In [1]:
import torch
import numpy as np

In [2]:
from utils.make_data import autoregressive
# from models.conformal import nonconformity, cover, ConformalForecaster

In [126]:
def generate_autoregressive_forecast_dataset(n_samples=100,
                                             seq_len=100,
                                             n_features=1,
                                             X_mean=1,
                                             X_variance=2,
                                             noise_profile=None,
                                             memory_factor=0.9,
                                             mode="time-dependent",
                                             horizon=10):
    total_seq_len = seq_len + horizon
    # Create the input features of the generating process
    X_gen = [np.random.normal(X_mean, X_variance, (total_seq_len,
                                                   n_features))
             for _ in range(n_samples)]
    w = np.array([memory_factor ** k for k in range(total_seq_len)])

    if noise_profile is None:
        # default increasing noise profile
        noise_profile = np.array(
            [1 / (seq_len - 1) * k for k in range(total_seq_len)])

    X = None  # X stores the time series values generated from features X_gen
    if mode == "noise-sweep":
        X = torch.FloatTensor(
            [[(autoregressive(X_gen[k], w).reshape(total_seq_len, n_features) +
               np.random.normal(0, noise_profile[u], (total_seq_len,
                                                      n_features)))
                  .reshape(total_seq_len, ) for k in range(n_samples)]
             for u in range(len(noise_profile))])


    elif mode == "time-dependent":
        X = torch.FloatTensor(
            [(autoregressive(X_gen[k], w)
              .reshape(total_seq_len, n_features) + (
                  torch.normal(mean=0.0, std=torch.tensor(noise_profile)))
              .detach().numpy().reshape(-1, n_features)).reshape(
                total_seq_len, )
                for k in range(n_samples)])

    # TODO clean up (un)squeezing.
    Y = torch.FloatTensor(X[:, -horizon:])  # `horizon` of predictions
    X = torch.nn.utils.rnn.pad_sequence(X[:, :-horizon],
                                        batch_first=True).unsqueeze(dim=-1)
    
    print(X.size(), Y.size())

    dataset = torch.utils.data.TensorDataset(X, Y)
    return dataset


In [153]:
import numpy as np
import torch
from torch import nn


def nonconformity(output, target):
    """Measures the nonconformity between output and target time series."""
    # Average MAE loss for every step in the sequence.
    return torch.nn.functional.l1_loss(output, target, reduction='none')


def cover(pred, target):
    # Returns True when the entire forecast fits into predicted conformal
    # intervals.
    # TODO joint vs independent coverage
    return torch.all(
        torch.logical_and(target >= pred[:, 0], target <= pred[:, 1])).item()


class ConformalForecaster(nn.Module):
    def __init__(self, embedding_size, input_size=1, output_size=1, horizon=1,
                 error_rate=0.05):
        super(ConformalForecaster, self).__init__()
        # input_size indicates the number of features in the time series
        # input_size=1 for univariate series.

        # Encoder and forecaster can be the same (if embeddings are
        # trained on `horizon`-step forecasts), but different models are
        # possible.

        # TODO try separate encoder and forecaster models.
        # TODO try the RNN autoencoder trained on reconstruction error.
        self.encoder = None

        self.forecaster_rnn = nn.LSTM(input_size=input_size,
                                      hidden_size=embedding_size,
                                      batch_first=True)
        self.forecaster_out = nn.Linear(embedding_size, output_size)

        self.horizon = horizon
        self.alpha = error_rate

        self.num_train = None
        self.calibration_scores = None
        self.critical_calibration_scores = None

    def forward(self, x, len_x):
        # len_x : torch.LongTensor
        # 		  Length of sequences (b, )
        sorted_len, idx = len_x.sort(dim=0, descending=True)
        sorted_x = x[idx]

        # Convert to packed sequence batch
        packed_x = torch.nn.utils.rnn.pack_padded_sequence(sorted_x,
                                                           lengths=sorted_len,
                                                           batch_first=True)

        # [batch, seq_len, embedding_size]
        packed_h, _ = self.forecaster_rnn(packed_x)

        max_seq_len = x.size(1)
        padded_output, _ = torch.nn.utils.rnn.pad_packed_sequence(packed_h,
                                                                  batch_first=True,
                                                                  total_length=max_seq_len)

        _, reverse_idx = idx.sort(dim=0, descending=False)
        padded_output = padded_output[reverse_idx]

        # [batch, horizon, output_size, 1]
        return self.forecaster_out(
            padded_output[:, -self.horizon:, :]).unsqueeze(-1)

    def fit(self, dataset, calibration_dataset, epochs, lr, batch_size=150):
        # Train the forecaster to return correct multi-step predictions.
        train_loader = torch.utils.data.DataLoader(dataset,
                                                   batch_size=batch_size,
                                                   shuffle=True)
        self.num_train = len(dataset)

        optimizer = torch.optim.Adam(self.parameters(), lr=lr)
        criterion = torch.nn.MSELoss()

        for epoch in range(epochs):
            self.train()
            train_loss = 0.

            for sequences, targets in train_loader:  # iterate through batches
                optimizer.zero_grad()

                out = self(sequences)

                loss = criterion(out, targets)
                loss.backward()

                train_loss += loss.item()

                optimizer.step()

            mean_train_loss = train_loss / len(train_loader)
            if epoch % 50 == 0:
                print(
                    'Epoch: {}\tTrain loss: {}'.format(epoch, mean_train_loss))

        # Collect calibration scores
        self.calibrate(calibration_dataset)

    def calibrate(self, calibration_dataset):
        """
        Computes the nonconformity scores for the calibration dataset.
        """
        calibration_loader = torch.utils.data.DataLoader(calibration_dataset,
                                                         batch_size=1)
        calibration_scores = []

        with torch.set_grad_enabled(False):
            self.eval()
            for sequences, targets in calibration_loader:
                out = self(sequences)
                calibration_scores.extend(
                    nonconformity(out, targets).detach().numpy())

        self.calibration_scores = torch.tensor(calibration_scores).T

        # Given p_{z}:=\frac{\left|\left\{i=m+1, \ldots, n+1: R_{i} \geq R_{n+1}\right\}\right|}{n-m+1}
        # and the accepted R_{n+1} = \Delta(y, f(x_{test})) are such that
        # p_{z} > \alpha we have that the nonconformity scores should be below
        # the (corrected) (1 - alpha)% of calibration scores.

        # TODO check: By applying (3) to Zcal, we get the sequence of
        # non-conformity scores and then sort them in descending order
        # α1, . . . , αq. Then, depending on the significance level ε, we define
        # the index of the (1 − ε)-percentile non-conformity score, αs, such as
        # s = ⌊ε(q + 1)⌋.
        self.critical_calibration_scores = torch.tensor([np.quantile(
            position_calibration_scores, q=1 - self.alpha * self.num_train / (
                    self.num_train + 1))
            for position_calibration_scores in self.calibration_scores])

    def predict(self, x):
        """Forecasts the time series with conformal uncertainty intervals."""
        out = self(x).squeeze()
        # TODO +/- nonconformity will not return *adaptive* interval widths.
        # TODO correction for multiple comparisons for each multi-horizon step.
        return torch.vstack([out - self.critical_calibration_scores,
                             out + self.critical_calibration_scores]).T


In [127]:
train_dataset = generate_autoregressive_forecast_dataset(n_samples=1000, seq_len=100, horizon=10)
calibration_dataset = generate_autoregressive_forecast_dataset(n_samples=1000, seq_len=100, horizon=10)
test_dataset = generate_autoregressive_forecast_dataset(n_samples=100, seq_len=100, horizon=10)

torch.Size([1000, 100, 1]) torch.Size([1000, 10])
torch.Size([1000, 100, 1]) torch.Size([1000, 10])
torch.Size([100, 100, 1]) torch.Size([100, 10])


In [ ]:
# X: [n_samples, max_seq_len, n_features]
# Y: [n_samples, horizon]

In [6]:
model = ConformalForecaster(embedding_size=8, horizon=10, error_rate=0.05)

In [7]:
model.fit(train_dataset, calibration_dataset, epochs=10, lr=0.01, batch_size=100)

TypeError: forward() missing 1 required positional argument: 'len_x'

In [42]:
model.critical_calibration_scores

tensor([9.4111, 9.8009, 9.6471, 9.4183, 9.5524, 9.5123, 9.4077, 9.2933, 9.5192,
        9.4380], dtype=torch.float64)

In [46]:
model.eval()
c = []
for sequence, target in test_dataset:
    sequence = sequence.unsqueeze(dim=0)
    pred = model.predict(sequence)
    c.append(cover(pred, target))
print('Achieved coverage: {}'.format(np.mean(c)))

Achieved coverage: 0.77


In [45]:
def cover(pred, target):
    # Returns True when the entire forecast fits into predicted conformal
    # intervals.
    return torch.all(
        torch.logical_and(target >= pred[:, 0], target <= pred[:, 1])).item()

In [37]:
for sequence, target in test_dataset:
    sequence = sequence.unsqueeze(dim=0)
    h, _ = model.forecaster_rnn(sequence)
    print(h.size())
    # [batch, horizon, output_size, 1]
    out = model.forecaster_out(h[:, -10:, :]).unsqueeze(-1)
    print(out.size())
    print(model.critical_calibration_scores.size())
    print(out.squeeze().size())
    
    break

torch.Size([1, 100, 8])
torch.Size([1, 10, 1, 1])
torch.Size([10])
torch.Size([10])


In [164]:
alpha = 0.05
num_train = 1000

def calibrate(calibration_dataset):
    """
    Computes the nonconformity scores for the calibration dataset.
    """
    calibration_loader = torch.utils.data.DataLoader(calibration_dataset,
                                                     batch_size=1)
    calibration_scores = []

    with torch.set_grad_enabled(False):
        model.eval()
        for sequences, targets in calibration_loader:
            out = model(sequences)
            calibration_scores.extend(nonconformity(out, targets).detach().numpy())

    calibration_scores = torch.tensor(calibration_scores).T

    # Given p_{z}:=\frac{\left|\left\{i=m+1, \ldots, n+1: R_{i} \geq R_{n+1}\right\}\right|}{n-m+1}
    # and the accepted R_{n+1} = \Delta(y, f(x_{test})) are such that
    # p_{z} > \alpha we have that the nonconformity scores should be below
    # the (corrected) alpha% of calibration scores.
    critical_calibration_scores = torch.tensor([np.quantile(
        position_calibration_scores, q= alpha * num_train / (num_train + 1))
        for position_calibration_scores in calibration_scores])
    print(critical_calibration_scores)

In [165]:
calibrate(calibration_dataset)

tensor([0.1335, 0.2231, 0.2458, 0.2786, 0.2465, 0.2878, 0.3393, 0.3049, 0.2977,
        0.3465], dtype=torch.float64)


In [176]:
1 - alpha * num_train / (num_train + 1)

0.9500499500499501

In [133]:
import numpy as np

def autoregressive(X_gen, w):
    """ Generates a single time series example. """
    return np.array(
        [np.sum(X_gen[0:k + 1] * np.flip(w[0:k + 1]).reshape(-1, 1)) for k in
         range(len(X_gen))])


seq_len = 10
horizon = 10
n_samples = 100
X_mean = 1
X_variance = 2
n_features = 1
memory_factor = 0.9

# sequence_lengths = [seq_len + horizon] * n_samples
sequence_lengths = range(horizon + 1, n_samples + horizon + 1)
max_seq_len = np.max(sequence_lengths)

# Create the input features of the generating process
X_gen = [np.random.normal(X_mean, X_variance, (seq_len,
                                               n_features))
         for seq_len in sequence_lengths]

# TODO determine how do weights and noise profile change depending on the
# length of series.
w = np.array([memory_factor ** k for k in range(max_seq_len)])

In [134]:
noise_profiles = [[1 / (seq_len - 1) * k for k in range(max(1, seq_len))] for seq_len in sequence_lengths]

In [84]:
X_gen[2].shape

(13, 1)

In [135]:
noise = [np.random.normal(0., noise_profile).reshape(-1, n_features) for noise_profile in noise_profiles]

In [136]:
ar = [autoregressive(X_gen[k], w).reshape(sequence_lengths[k], n_features) for k in range(n_samples)]

In [137]:
X_full = [torch.tensor(i + j) for i, j in zip(ar, noise)]

In [138]:
len(X_full)

100

In [139]:
X[0].shape

torch.Size([1, 1])

In [90]:
[autoregressive(X_gen[k], w).reshape(-1, n_features) for k in range(n_samples)][0].shape

(11, 1)

In [92]:
X_full = X

In [140]:
X, Y = [], []
for seq in X_full:
    seq_len = len(seq)
    if seq_len >= 2 * horizon:
        X.append(seq[:-horizon])
        Y.append(seq[-horizon:])
    elif seq_len > horizon:
        X.append(seq[:seq_len - horizon])
        Y.append(seq[-(seq_len - horizon):])

In [143]:
# for x in X:
print(len(X[99]))

100


In [144]:
for i in range(len(X)):
    print('X_full: {}\tX: {}\tY: {}'.format(len(X_full[i]), len(X[i]), len(Y[i])))

X_full: 11	X: 1	Y: 1
X_full: 12	X: 2	Y: 2
X_full: 13	X: 3	Y: 3
X_full: 14	X: 4	Y: 4
X_full: 15	X: 5	Y: 5
X_full: 16	X: 6	Y: 6
X_full: 17	X: 7	Y: 7
X_full: 18	X: 8	Y: 8
X_full: 19	X: 9	Y: 9
X_full: 20	X: 10	Y: 10
X_full: 21	X: 11	Y: 10
X_full: 22	X: 12	Y: 10
X_full: 23	X: 13	Y: 10
X_full: 24	X: 14	Y: 10
X_full: 25	X: 15	Y: 10
X_full: 26	X: 16	Y: 10
X_full: 27	X: 17	Y: 10
X_full: 28	X: 18	Y: 10
X_full: 29	X: 19	Y: 10
X_full: 30	X: 20	Y: 10
X_full: 31	X: 21	Y: 10
X_full: 32	X: 22	Y: 10
X_full: 33	X: 23	Y: 10
X_full: 34	X: 24	Y: 10
X_full: 35	X: 25	Y: 10
X_full: 36	X: 26	Y: 10
X_full: 37	X: 27	Y: 10
X_full: 38	X: 28	Y: 10
X_full: 39	X: 29	Y: 10
X_full: 40	X: 30	Y: 10
X_full: 41	X: 31	Y: 10
X_full: 42	X: 32	Y: 10
X_full: 43	X: 33	Y: 10
X_full: 44	X: 34	Y: 10
X_full: 45	X: 35	Y: 10
X_full: 46	X: 36	Y: 10
X_full: 47	X: 37	Y: 10
X_full: 48	X: 38	Y: 10
X_full: 49	X: 39	Y: 10
X_full: 50	X: 40	Y: 10
X_full: 51	X: 41	Y: 10
X_full: 52	X: 42	Y: 10
X_full: 53	X: 43	Y: 10
X_full: 54	X: 44	Y: 10
X_full

In [145]:
len(X_full)

100

In [100]:
len(X_gen)

7

In [148]:
X_tensor = torch.nn.utils.rnn.pad_sequence(X, batch_first=True)

In [149]:
X_tensor.size()

torch.Size([100, 100, 1])

In [150]:
Y_tensor = torch.nn.utils.rnn.pad_sequence(Y, batch_first=True)

In [151]:
Y_tensor.size()

torch.Size([100, 10, 1])

In [ ]:
# X: [n_samples, max_seq_len, n_features]
# Y: [n_samples, horizon]

In [155]:
torch.LongTensor(sequence_lengths).size()

torch.Size([100])

In [158]:
torch.utils.data.TensorDataset(X_tensor, Y_tensor)